! alle Pakete installieren

# Pakete importieren 

In [ ]:
import random
import statistics

# Code

## 1) Parameter des Systems

In [ ]:
LAMBDA_ANKUNFT_PRO_WOCHE = 20   # Ø neue Anmeldungen pro Woche
SIMULATIONS_DAUER_WOCHEN = 200  # über wie viele Wochen simuliert wird

# Kapazität
ANZAHL_THERAPEUTEN = 5          # Anzahl Therapeut*innen

# Therapiedauer (Servicezeit) in Wochen
MIN_THERAPIEDAUER = 10          # z. B. 10 Sitzungen à 1 Woche
MAX_THERAPIEDAUER = 40          # z. B. 40 Sitzungen

# Zufalls-Seed für Reproduzierbarkeit (ändern(?))
random.seed(42)

## 2) Funktionen

In [ ]:
def generiere_ankunftszeiten(lambda_pro_woche, sim_dauer_wochen):
    """
    Erzeugt eine Liste aufsteigender Ankunftszeiten (in Wochen)
    auf Basis eines Poisson-Prozesses (exponentielle Zwischenankunftszeiten).
    """
    ankunftszeiten = []
    aktuelle_zeit = 0.0

    # mittlere Zwischenankunftszeit
    mean_interarrival = 1.0 / lambda_pro_woche  # Wochen

    while aktuelle_zeit < sim_dauer_wochen:
        # Exponentiell verteilte Zwischenankunftszeit
        delta = random.expovariate(1.0 / mean_interarrival)
        aktuelle_zeit += delta
        if aktuelle_zeit > sim_dauer_wochen:
            break
        ankunftszeiten.append(aktuelle_zeit)

    return ankunftszeiten


def ziehe_therapiedauer(min_wochen, max_wochen):
    """
    Ziehe eine zufällige Therapiedauer (in Wochen).
    Hier: einfache Gleichverteilung zwischen min und max.
    Später kannst du z.B. Normal- oder andere Verteilungen verwenden.
    """
    return random.uniform(min_wochen, max_wochen)


## 3) Zielgrößen

## 4) Simulationslauf

In [ ]:
def simuliere_warteschlange():
    # 3.1 Ankunftszeiten generieren
    ankunftszeiten = generiere_ankunftszeiten(
        LAMBDA_ANKUNFT_PRO_WOCHE,
        SIMULATIONS_DAUER_WOCHEN
    )

    # nächster Zeitpunkt, an dem jede/r Therapeut*in frei ist
    # Index: Therapeut, Wert: Zeit (in Wochen)
    naechst_frei = [0.0 for _ in range(ANZAHL_THERAPEUTEN)]

    # Listen zur Auswertung
    wartezeiten = []     # nur echte Wartezeit (bis Therapiebeginn)
    systemzeiten = []    # Zeit von Anmeldung bis Therapieende

    # zur Auslastungsberechnung: gesamte Behandlungszeit je Therapeut
    gesamt_behandlungszeit_pro_therapeut = [0.0 for _ in range(ANZAHL_THERAPEUTEN)]

    for ankunft in ankunftszeiten:
        # Therapiedauer für diesen Patienten
        servicezeit = ziehe_therapiedauer(
            MIN_THERAPIEDAUER, MAX_THERAPIEDAUER
        )

        # 3.2 Finde den Therapeuten, der am frühesten frei ist
        therapeut_index = min(range(ANZAHL_THERAPEUTEN),
                              key=lambda i: naechst_frei[i])
        frueheste_frei_zeit = naechst_frei[therapeut_index]

        # 3.3 Therapiebeginn: max(Ankunft, frühester Frei-Zeitpunkt)
        therapiebeginn = max(ankunft, frueheste_frei_zeit)
        therapieende = therapiebeginn + servicezeit

        # 3.4 Warte- und Systemzeit berechnen
        wartezeit = therapiebeginn - ankunft
        systemzeit = therapieende - ankunft

        wartezeiten.append(wartezeit)
        systemzeiten.append(systemzeit)

        # 3.5 Kalender des Therapeuten aktualisieren
        naechst_frei[therapeut_index] = therapieende
        gesamt_behandlungszeit_pro_therapeut[therapeut_index] += servicezeit

    # ============================================
    # 4. Kennzahlen berechnen
    # ============================================

    # Durchschnittliche Wartezeit & Systemzeit
    mittlere_wartezeit = statistics.mean(wartezeiten) if wartezeiten else 0.0
    mittlere_systemzeit = statistics.mean(systemzeiten) if systemzeiten else 0.0

    # Median & ein paar Quantile (optional)
    wartezeiten_sortiert = sorted(wartezeiten)
    def quantil(values, q):
        if not values:
            return 0.0
        idx = int(q * (len(values) - 1))
        return values[idx]

    median_warte = quantil(wartezeiten_sortiert, 0.5)
    q75_warte = quantil(wartezeiten_sortiert, 0.75)
    q90_warte = quantil(wartezeiten_sortiert, 0.90)

    # Auslastung der Therapeut*innen
    # Annahme: Simulation läuft SIMULATIONS_DAUER_WOCHEN,
    # und jede/r Therapeut*in könnte theoretisch durchgehend arbeiten.
    auslastungen = [
        behandlungszeit / SIMULATIONS_DAUER_WOCHEN
        for behandlungszeit in gesamt_behandlungszeit_pro_therapeut
    ]
    mittlere_auslastung = statistics.mean(auslastungen) if auslastungen else 0.0

    # Anteil mit Wartezeit > X Wochen (z. B. 12 und 24 Wochen)
    def anteil_mit_wartezeit_groesser_x(x):
        if not wartezeiten:
            return 0.0
        return sum(w > x for w in wartezeiten) / len(wartezeiten)

    anteil_ueber_12 = anteil_mit_wartezeit_groesser_x(12)
    anteil_ueber_24 = anteil_mit_wartezeit_groesser_x(24)

## 5) Kennzahlen berechnen 

In [ ]:
# Durchschnittliche Wartezeit & Systemzeit
    mittlere_wartezeit = statistics.mean(wartezeiten) if wartezeiten else 0.0
    mittlere_systemzeit = statistics.mean(systemzeiten) if systemzeiten else 0.0

    # Median & ein paar Quantile (optional)
    wartezeiten_sortiert = sorted(wartezeiten)
    def quantil(values, q):
        if not values:
            return 0.0
        idx = int(q * (len(values) - 1))
        return values[idx]

    median_warte = quantil(wartezeiten_sortiert, 0.5)
    q75_warte = quantil(wartezeiten_sortiert, 0.75)
    q90_warte = quantil(wartezeiten_sortiert, 0.90)

    # Auslastung der Therapeut*innen
    # Annahme: Simulation läuft SIMULATIONS_DAUER_WOCHEN,
    # und jede/r Therapeut*in könnte theoretisch durchgehend arbeiten.
    auslastungen = [
        behandlungszeit / SIMULATIONS_DAUER_WOCHEN
        for behandlungszeit in gesamt_behandlungszeit_pro_therapeut
    ]
    mittlere_auslastung = statistics.mean(auslastungen) if auslastungen else 0.0

    # Anteil mit Wartezeit > X Wochen (z. B. 12 und 24 Wochen)
    def anteil_mit_wartezeit_groesser_x(x):
        if not wartezeiten:
            return 0.0
        return sum(w > x for w in wartezeiten) / len(wartezeiten)

    anteil_ueber_12 = anteil_mit_wartezeit_groesser_x(12)
    anteil_ueber_24 = anteil_mit_wartezeit_groesser_x(24)

## 6) Ergbenisse ausgeben

In [ ]:
print("=== Ergebnisse der Simulation (Therapie-Warteschlange) ===")
    print(f"Anzahl Therapeut*innen:            {ANZAHL_THERAPEUTEN}")
    print(f"Simulierte Zeit:                   {SIMULATIONS_DAUER_WOCHEN:.1f} Wochen")
    print(f"Anzahl Anmeldungen:                {len(ankunftszeiten)}")

    print("\nWartezeiten (in Wochen)")
    print(f"  Mittelwert:                      {mittlere_wartezeit:.2f}")
    print(f"  Median:                          {median_warte:.2f}")
    print(f"  75%-Quantil:                     {q75_warte:.2f}")
    print(f"  90%-Quantil:                     {q90_warte:.2f}")
    print(f"  Anteil > 12 Wochen:              {anteil_ueber_12*100:.1f} %")
    print(f"  Anteil > 24 Wochen:              {anteil_ueber_24*100:.1f} %")

    print("\nGesamtzeit im System (Anmeldung bis Therapieende)")
    print(f"  Mittelwert:                      {mittlere_systemzeit:.2f} Wochen")

    print("\nAuslastung der Therapeut*innen")
    for i, rho in enumerate(auslastungen):
        print(f"  Therapeut {i+1}:                 {rho*100:.1f} %")
    print(f"  Durchschnittliche Auslastung:    {mittlere_auslastung*100:.1f} %")
    
    if __name__ == "__main__":
    simuliere_warteschlange()

# Komplett 

"""
Einfache SimPy-Simulation für Verhaltenstherapie (VT)

Annahmen:
- 1 Sitzung = 50 Minuten
- 14 600 Therapeut*innen
- Jede Person kann 35 Sitzungen / Woche geben
  → wir interpretieren das als 35 Patient*innen parallel (1 Sitzung/Woche)
- Therapiedauer: 24 Sitzungen → 24 Wochen
- Ankünfte: Poisson-Prozess mit Rate ARRIVAL_RATE_PER_WEEK (neue Patient*innen/Woche)
- Zeitmaß: Wochen
"""

In [ ]:
import simpy
import numpy as np
import statistics as stats

# ==========================
# Parameter
# ==========================

# Struktur / Kapazität
NUM_THERAPISTS = 6_230
SESSIONS_PER_WEEK_PER_THERAPIST = 35          # voll ausgelastet
SLOTS_PER_THERAPIST = SESSIONS_PER_WEEK_PER_THERAPIST

TOTAL_CAPACITY = NUM_THERAPISTS * SLOTS_PER_THERAPIST  # parallele Patient*innen

# Therapie-Eigenschaften
SESSION_DURATION_MIN = 50                     # Info, im Modell nicht direkt genutzt
NUM_SESSIONS_PER_THERAPY = 24                 # Länge der Therapie
THERAPY_DURATION_WEEKS = NUM_SESSIONS_PER_THERAPY      # 1 Sitzung/Woche

# Nachfrage: Ankünfte pro Woche
# Diesen Wert solltest du später mit einer realistischen Schätzung ersetzen.
ARRIVAL_RATE_PER_WEEK = 20_000               # neue Patient*innen pro Woche (Annahme!)
MEAN_INTERARRIVAL = 1.0 / ARRIVAL_RATE_PER_WEEK

# Simulation
# 18_000_000 * 0,28 = 5_040_000
# Perfekte Anzahl an Patienten 10_000
NUM_PATIENTS = 5_000_000                        # wie viele Patient*innen wir simulieren
RANDOM_SEED = 42


# ==========================
# Prozesse
# ==========================

def patient_process(env, name, resource, wait_times, system_times):
    """Prozess für eine Patient*in."""
    arrival_time = env.now

    # Therapieplatz anfordern
    with resource.request() as req:
        yield req
        start_time = env.now
        wait = start_time - arrival_time
        wait_times.append(wait)

        # Therapie: feste Dauer (24 Wochen)
        yield env.timeout(THERAPY_DURATION_WEEKS)
        end_time = env.now
        system_times.append(end_time - arrival_time)


def arrival_process(env, resource, wait_times, system_times, rng):
    """Erzeugt nach und nach neue Patient*innen mit exponentiellen Zwischenankünften."""
    for i in range(NUM_PATIENTS):
        interarrival = rng.exponential(MEAN_INTERARRIVAL)
        yield env.timeout(interarrival)
        env.process(patient_process(env, f"Patient {i}", resource, wait_times, system_times))


# ==========================
# Simulation
# ==========================

def run_simulation():
    rng = np.random.default_rng(RANDOM_SEED)
    env = simpy.Environment()

    # Ressource: alle Therapieplätze
    therapy_slots = simpy.Resource(env, capacity=TOTAL_CAPACITY)

    wait_times = []
    system_times = []

    # Ankunftsprozess starten
    env.process(arrival_process(env, therapy_slots, wait_times, system_times, rng))

    # Simulation laufen lassen (bis alle Patienten fertig sind)
    env.run()

    if not wait_times:
        print("Keine Patient*innen simuliert – prüfe NUM_PATIENTS.")
        return

    # Kennzahlen
    mean_wait = stats.mean(wait_times)
    median_wait = stats.median(wait_times)
    q75 = float(np.quantile(wait_times, 0.75))
    q90 = float(np.quantile(wait_times, 0.90))

    share_over_12 = sum(w > 12 for w in wait_times) / len(wait_times)
    share_over_24 = sum(w > 24 for w in wait_times) / len(wait_times)

    mean_system = stats.mean(system_times)

    # Ausgabe
    print("=== SimPy-Simulation: Verhaltenstherapie (einfaches Modell) ===")
    print(f"Therapeut*innen gesamt:           {NUM_THERAPISTS:,}")
    print(f"Plätze pro Therapeut*in:          {SLOTS_PER_THERAPIST}")
    print(f"Gesamtkapazität (Slots):          {TOTAL_CAPACITY:,}")
    print(f"Therapiedauer:                    {THERAPY_DURATION_WEEKS} Wochen")
    print(f"Ankunftsrate:                     {ARRIVAL_RATE_PER_WEEK:,} neue Patient*innen/Woche")
    print(f"Simulierte Patient*innen:         {NUM_PATIENTS:,}\n")

    print("Wartezeit bis Therapiebeginn (in Wochen):")
    print(f"  Mittelwert:                     {mean_wait:.2f}")
    print(f"  Median:                         {median_wait:.2f}")
    print(f"  75%-Quantil:                    {q75:.2f}")
    print(f"  90%-Quantil:                    {q90:.2f}")
    print(f"  Anteil > 12 Wochen:             {share_over_12*100:.1f} %")
    print(f"  Anteil > 24 Wochen:             {share_over_24*100:.1f} %\n")

    print("Gesamtzeit im System (Anmeldung → Therapieende):")
    print(f"  Mittelwert:                     {mean_system:.2f} Wochen")


if __name__ == "__main__":
    run_simulation()

=== SimPy-Simulation: Verhaltenstherapie (einfaches Modell) ===
Therapeut*innen gesamt:           14,600
Plätze pro Therapeut*in:          35
Gesamtkapazität (Slots):          511,000
Therapiedauer:                    24 Wochen
Ankunftsrate:                     20,000 neue Patient*innen/Woche
Simulierte Patient*innen:         5,000,000

Wartezeit bis Therapiebeginn (in Wochen):
  Mittelwert:                     0.00
  Median:                         0.00
  75%-Quantil:                    0.00
  90%-Quantil:                    0.00
  Anteil > 12 Wochen:             0.0 %
  Anteil > 24 Wochen:             0.0 %

Gesamtzeit im System (Anmeldung → Therapieende):
  Mittelwert:                     24.00 Wochen


## a)